In [93]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

In [94]:
# B92 Protocol
n = 500
np.random.seed(seed=0)
# Alice chooses a random bit string (information she intend to use in Secure Key), this 
# infomration remains private to her.
alice_bits = randint(2, size=n)
print(alice_bits)  # note there is no need to generate random basis for measurements unlike BB84

[0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0
 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1
 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1
 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0
 0 1 1 1 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0
 0 1 1 0 0 0 1 1 0 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1 0 0
 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1
 1 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1
 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 1 0
 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0
 1 0 1 0 0 0 0 1 1 1 1 1 

In [95]:
def encode_message_B92(bits):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bits[i] == 0: # for bit value = 0, prepare photon in Horizontal polarization or Qubit state = |0⟩ 
            pass
        else: # for bit value = 1 Prepare photon as +45D polarized or Qubit state  = |+⟩
            qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [96]:
message = encode_message_B92(alice_bits)

In [97]:
# Let's check the a message bit encoded in  quantum information
print('bit = %i' % alice_bits[0])
message[0].draw()


bit = 0


░ 
q_0: ─░─
      ░ 
c: 1/═══

In [98]:
# Assume our quantum information has travelled to other end, now Bob has to chose from Rectiliear or Diagonal 
# bases to measure each photon sent by Alice that has travlled thru channel to Bob in real setup Bob has to just
# check if a photon is detected in a detector oriented Vertically or -45D  
bob_bases = randint(2, size=n)
print(bob_bases)

[1 1 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 1
 1 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1
 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1
 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 0 0
 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0
 0 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 1
 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0
 0 1 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 0 1
 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1
 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 1 1
 1 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0
 1 1 1 0 0 1 0 1 1 0 0 0 

In [99]:
def measure_message_B92(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # if random choice of basis is to measure in Z-basis
            message[q].measure(0, 0)
        if bases[q] == 1: # if random choice of basis is to measure in X-basis
            message[q].h(0)
            message[q].measure(0, 0)
        aer_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(2)  # by default assign a value of 2 to discard bit if no photon is detected in |V⟩ or |-⟩ state  
        if (bases[q] == 0 and measured_bit == 1) : # if measured in R basis and outcome is 1 it is |V⟩ state, bitvalue = 1
            measurements[q] = measured_bit
        if (bases[q] == 1 and measured_bit == 1) : # if measured in D basis and outcome is 1 it is |-⟩ state bitvalue = 0
            measurements[q] = 0

    return measurements

In [100]:
bob_results = measure_message_B92(message, bob_bases)

In [101]:
# check how it has processed for couple of bits coded as Qubits in entire flow circuit.
message[0].draw()

░ ┌───┐┌─┐
q_0: ─░─┤ H ├┤M├
      ░ └───┘└╥┘
c: 1/═════════╩═
              0

In [102]:
message[6].draw()

┌───┐ ░ ┌───┐┌─┐
q_0: ┤ H ├─░─┤ H ├┤M├
     └───┘ ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

In [103]:
print(bob_results) # results private to bob

[2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 0, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 1, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 1, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 

In [104]:
def get_good_bits(ab_bits, m_bits): # remove bits where no photons are detected
    good_bits = []
    for q in range(n):
        if m_bits[q] != 2:
            good_bits.append(ab_bits[q])
    return good_bits

In [105]:
# share information with alice on bits selected tp forma key
alice_key = get_good_bits(alice_bits, bob_results)
print(alice_key)
bob_key = get_good_bits(bob_results, bob_results)
print(bob_key)
print (" \nNumber of good bits ", len(bob_key))

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1]
 
Number of good bits  121


In [106]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [107]:
# Take a sample from bits that has macthed the bases 
sample_size = 25
key_size = len(bob_key)
bit_selection = randint(key_size, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

  bob_sample = [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1]
alice_sample = [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1]


In [110]:
print(bob_key)
print(alice_key)
print("\nFinal Secure key length = %i" % len(alice_key))

[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]

Final Secure key length = 96


In [112]:
bob_sample == alice_sample

True

In [126]:
# Run entire program at once
np.random.seed(seed=3)
## Step 1, prepare random bits
alice_bits = randint(2, size=n)
message = encode_message_B92(alice_bits)
## Step 2, Interception!! optional, comment below two statements to remove Eve's presence
#eve_bases = randint(2, size=n)
#intercepted_message = measure_message_B92(message, eve_bases)
## Step 3, Bob generates Random Bases
bob_bases = randint(2, size=n)
bob_results = measure_message_B92(message, bob_bases)
## Step 4
bob_key = get_good_bits(alice_bits, bob_results)
alice_key = get_good_bits(bob_results, bob_results)
## Step 5
sample_size = 25
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

  bob_sample = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0]
alice_sample = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0]


In [127]:
# check if sample bits from Alice and Bob's key are matching? 
bob_sample == alice_sample

True

In [125]:
# End of B92 Protocol program using qiskit